### Computer Network Example
- Given an IP network where we consider only one direction of transmission indicated by arrows.
- The number on each link indicates the time taken by an IP packet (in seconds) to be transferred over that link.
- What is the fastest path to forward an IP packet from router A to router G?


<img src="image01.png" alt="Network diagram showing routers A through G with directional links and transmission times" width="400" height="auto">

### Decision Variables

$x_{ij} \in \{0,1\}$ for all links $(i,j)$ in the network

where:
- $x_{ij} = 1$ if link from node $i$ to node $j$ is used in the path
- $x_{ij} = 0$ otherwise


### Objective Function

Minimize the total time (cost) of the path from A to G:

$\min Z = \sum_{(i,j)\in\mathcal{A}} d_{ij} X_{ij}$

where  
- $\mathcal{A}$ is the set of directed arcs,  
- $d_{a}$ is the cost on arc $a$,  
- $x_{a} \in \{0,1\}$ indicates whether arc $a$ is used.


### Constraints

1. Starting point: packet must leave router A:

   $X_{AB} + X_{AC} = 1$

2. Ending point: packet must enter router G:

   $X_{DG} + X_{EG} = 1$

3. Flow conservation (for every intermediate node):
   - node B: $X_{AB} + X_{CB} = X_{BD} + X_{BE}$  
   - node C: $X_{AC} = X_{CB} + X_{CE} + X_{CF}$  
   - node D: $X_{BD} = X_{DE} + X_{DG}$  
   - node E: $X_{BE} + X_{CE} + X_{DE} + X_{FE} = X_{EG}$  
   - node F: $X_{CF} = X_{FE}$
   

In [30]:
#import all the needed library
import gurobipy as gp
from gurobipy import GRB

In [31]:
#make a dictionary of arcs and their costs
arcs, cost = gp.multidict({
    ('A', 'B'): 4,('A', 'C'): 3,
    ('B', 'D'): 6,('B', 'E'): 5,
    ('C', 'B'): 3,('C', 'E'): 4,('C', 'F'): 6,
    ('D', 'G'): 1,('D', 'E'): 2,
    ('F', 'E'): 6,
    ('E', 'G'): 3,
      })


In [32]:
#create the model

model = gp.Model('Computer_Network_fastest_path')

#create the decision variables
X = model.addVars(arcs, vtype= GRB.BINARY, name='X')

In [33]:
#objective function
model.setObjective(sum(cost[a]* X[a] for a in arcs), GRB.MINIMIZE)

In [34]:
#add the constraints

#source node constraint
model.addConstr(X.sum('A', '*') == 1, 'source')

#destination node constraint
model.addConstr(X.sum('*', 'G') == 1, 'destination')

#flow conservation constraints
for n in ['B', 'C', 'D', 'E', 'F']:
    model.addConstr(X.sum('*', n) == X.sum(n, '*'))
    #model.addConstr(X.sum('n', '*') - x.sum('*', 'n') == 0)


In [35]:
#turn off the solver output
gp.setParam('OutputFlag', 0)
#optimize the model
model.optimize()
#print the results
path = [ a for a in arcs if X[a].X > 0.5]
print('The fastest path from A to G is:', path, 
      '\nthe total cost is:', model.objVal)

The fastest path from A to G is: [('A', 'C'), ('C', 'E'), ('E', 'G')] 
the total cost is: 10.0
